# [PROTOTYPE] - API Examples for Python Developers

## Introduction

Welcome to the [Python package](https://github.com/GlobalFishingWatch/gfw-api-python-client) for accessing data from [Global Fishing Watch APIs](https://globalfishingwatch.org/our-apis/documentation#introduction). The [gfw-api-python-client](https://github.com/GlobalFishingWatch/gfw-api-python-client) is a simple wrapper for the Global Fishing Watch (GFW) APIs. It provides convenient functions to freely pull GFW data.

Our APIs provide software instructions that enable automatic connection of our data and products to other systems so users can easily download and integrate our datasets, code, and models to power their own platforms. By providing streamlined access to information on apparent fishing vessel activity, identity and history, we are helping create solutions for a resilient, sustainable ocean.

Global Fishing Watch APIs are available for noncommercial use only in accordance with the [CC BY-NC 4.0 license](https://creativecommons.org/licenses/by-nc/4.0/) including complying with the attribution requirements set forth in Section 3 below. To see more details, please visit our [Terms of Use](https://globalfishingwatch.org/our-apis/documentation#terms-of-use). They are used by researchers, governments, technology companies, etc.

We combine tracking data from the publicly available automatic identification system (AIS) and integrate that with information acquired through vessel monitoring systems that are operated by governments and made available to us through partnerships that we secure. Both AIS and VMS combine global positioning with a transmitter to regularly broadcast vessel location.

## Import libraries

In [1]:
import datetime
import os
import logging

import gfwapiclient as gfw

## Instantiate GFW API client

In [2]:
api_base_url = os.getenv("GFW_API_BASE_URL")
api_access_token = os.getenv("GFW_API_ACCESS_TOKEN")

client = gfw.Client(
    api_base_url=api_base_url,
    api_access_token=api_access_token
)

## Map Visualization - 4Wings API

[4Wings API (aka Map Visualization)](https://globalfishingwatch.org/our-apis/documentation#map-visualization-4wings-api) allows you to get fast visualisation, navigation and analysis for gridded spatiotemporal datasets. You can:

- visualize apparent fishing effort or SAR vessel detections in png or mvt format
- generate a report for apparent fishing effort data and for SAR (Synthetic-aperture radar) vessel detections.


### Import necessary data types

In [3]:
from gfwapiclient.resources.fourwings.report.models.request import (
    FourWingsReportDataset,
    FourWingsReportGroupBy,
    FourWingsReportRegion,
    FourWingsReportSpatialResolution,
    FourWingsReportTemporalResolution,
)

### Create a report of a specified region

Generate a report for apparent fishing effort data and for SAR (Synthetic-aperture radar) vessel detections.

#### Define request parameters

In [4]:
spatial_resolution = FourWingsReportSpatialResolution.HIGH
group_by = FourWingsReportGroupBy.VESSEL_ID
temporal_resolution = FourWingsReportTemporalResolution.DAILY
datasets = [
    FourWingsReportDataset.FISHING_EFFORT_LATEST,
    FourWingsReportDataset.SAR_PRESENCE_LATEST,
]
filters = None
date_range = "2022-01-01,2022-01-15"  # TODO: use start_date, end_date
spatial_aggregation = None
geojson = None
region = FourWingsReportRegion(dataset="public-eez-areas", id="8371")

#### Generate report

In [5]:
wings_report = await client.fourwings.aget_report(
    spatial_resolution=spatial_resolution,
    group_by=group_by,
    temporal_resolution=temporal_resolution,
    datasets=datasets,
    filters=filters,
    date_range=date_range,
    spatial_aggregation=spatial_aggregation,
    geojson=geojson,
    region=region,
)

In [6]:
wings_report

In [7]:
wings_report.data[-1]

FourWingsReportItem(date='2022-01-05', detections=1, flag='SEN', gear_type='TRAWLERS', hours=None, vessel_ids=None, vessel_id='0e0369b43-36c3-f0a8-adda-370f600ed334', vessel_type='FISHING', entry_timestamp=datetime.datetime(2022, 1, 5, 19, 17, 58, tzinfo=TzInfo(UTC)), exit_timestamp=datetime.datetime(2022, 1, 10, 19, 26, 8, tzinfo=TzInfo(UTC)), first_transmission_date=datetime.datetime(2016, 2, 4, 15, 32, 18, tzinfo=TzInfo(UTC)), last_transmission_date=datetime.datetime(2025, 2, 28, 8, 56, 48, tzinfo=TzInfo(UTC)), imo='', mmsi='663146000', call_sign='6WLM', dataset='public-global-vessel-identity:v3.0', report_dataset='public-global-sar-presence:v3.0', ship_name='F/V AUDREY-', lat=12.45, lon=-17.389999389648438)

In [8]:
wings_report.data[-1].model_dump()

{'date': '2022-01-05',
 'detections': 1,
 'flag': 'SEN',
 'gear_type': 'TRAWLERS',
 'hours': None,
 'vessel_ids': None,
 'vessel_id': '0e0369b43-36c3-f0a8-adda-370f600ed334',
 'vessel_type': 'FISHING',
 'entry_timestamp': datetime.datetime(2022, 1, 5, 19, 17, 58, tzinfo=TzInfo(UTC)),
 'exit_timestamp': datetime.datetime(2022, 1, 10, 19, 26, 8, tzinfo=TzInfo(UTC)),
 'first_transmission_date': datetime.datetime(2016, 2, 4, 15, 32, 18, tzinfo=TzInfo(UTC)),
 'last_transmission_date': datetime.datetime(2025, 2, 28, 8, 56, 48, tzinfo=TzInfo(UTC)),
 'imo': '',
 'mmsi': '663146000',
 'call_sign': '6WLM',
 'dataset': 'public-global-vessel-identity:v3.0',
 'report_dataset': 'public-global-sar-presence:v3.0',
 'ship_name': 'F/V AUDREY-',
 'lat': 12.45,
 'lon': -17.389999389648438}

## Vessels API

[Vessels API](https://globalfishingwatch.org/our-apis/documentation#vessels-api) allows you to search for vessels and get vessel identity details. This API combines Global Fishing Watch (GFW) core AIS identity data with GFW registry database to enable an improved understanding of vessel identity over time.

You can:

- Search for AIS and across over 40 public international, regional and national vessel registries. Check list of sources in the [reference data section](https://globalfishingwatch.org/our-apis/documentation#vessel-api-registry-codes-data-sources)
- Get identity details based on AIS self reported data or public registries. Check more detail about data caveats [here](https://globalfishingwatch.org/our-apis/documentation#data-caveat).
  

### Import necessary data types

In [9]:
from gfwapiclient.resources.vessels.base.enums import VesselDataset

### Search vessels

#### Define request parameters

In [10]:
datasets = [VesselDataset.VESSEL_IDENTITY_LATEST]
query = "775998121"
where = 'ssvid="775998121" AND shipname="DON TITO"'

#### Search vessels

In [11]:
vessels = await client.vessels.asearch_vessels(query=query, datasets=datasets)

In [12]:
vessels

In [13]:
vessels.data[-1].registry_info

[]

In [14]:
vessels.data[-1].combined_sources_info

[CombinedSourceInfo(vessel_id='bae8f325c-cf0a-01fe-6d1a-9a275588d4ff', gear_types=[GearType(name='OTHER', source='COMBINATION_OF_REGISTRY_AND_AIS_INFERRED_NN_INFO', year_from=2021, year_to=2023)], ship_types=[ShipType(name='OTHER', source='COMBINATION_OF_REGISTRY_AND_AIS_INFERRED_NN_INFO', year_from=2021, year_to=2023)])]

In [15]:
vessels.data[-1].model_dump()

{'dataset': 'public-global-vessel-identity:v3.0',
 'registry_info_total_records': 0,
 'registry_info': [],
 'registry_owners': [],
 'registry_public_authorizations': [],
 'combined_sources_info': [{'vessel_id': 'bae8f325c-cf0a-01fe-6d1a-9a275588d4ff',
   'gear_types': [{'name': 'OTHER',
     'source': 'COMBINATION_OF_REGISTRY_AND_AIS_INFERRED_NN_INFO',
     'year_from': 2021,
     'year_to': 2023}],
   'ship_types': [{'name': 'OTHER',
     'source': 'COMBINATION_OF_REGISTRY_AND_AIS_INFERRED_NN_INFO',
     'year_from': 2021,
     'year_to': 2023}]}],
 'self_reported_info': [{'id': 'bae8f325c-cf0a-01fe-6d1a-9a275588d4ff',
   'ssvid': '775998121',
   'ship_name': 'DON TITO',
   'n_ship_name': 'DONTITO',
   'flag': 'VEN',
   'call_sign': None,
   'imo': None,
   'messages_counter': 54,
   'positions_counter': 38,
   'source_code': ['AIS'],
   'match_fields': 'NO_MATCH',
   'transmission_date_from': datetime.datetime(2021, 8, 3, 13, 41, 41, tzinfo=TzInfo(UTC)),
   'transmission_date_to': da

### Get list of vessels filtered by ids

#### Define request parameters

In [16]:
datasets = [VesselDataset.VESSEL_IDENTITY_LATEST]
ids = ["8c7304226-6c71-edbe-0b63-c246734b3c01"]

#### Get list of vessels by their ids

In [17]:
vessels = await client.vessels.aget_vessels(ids=ids, datasets=datasets)

In [18]:
vessels.data[-1].registry_info

[RegistryInfo(id='a8d00ce54b37add7f85a35fcce8e7a1b', source_code=['GFW-REVIEW', 'IMO', 'NEAFC', 'TMT_OTHER'], ssvid='273379740', flag='RUS', ship_name='FRIO FORWIN', n_ship_name='FRIOFORWIN', call_sign='UCRZ', imo='9076260', latest_vessel_info=True, transmission_date_from=datetime.datetime(2015, 2, 27, 10, 59, 43, tzinfo=TzInfo(UTC)), transmission_date_to=datetime.datetime(2025, 2, 27, 11, 22, 25, tzinfo=TzInfo(UTC)), gear_types=['SPECIALIZED_REEFER'], length_m=126.8, tonnage_gt=6963.6, vessel_info_reference='186aeab79b33d7f5560894c4ad53a0b1', extra_fields=[ExtraField(registry_source='TMT', iuu_status=None, has_compliance_info=None, images=[], operator=None, builtYear=None, depthM=None)]),
 RegistryInfo(id='a8d00ce54b37add7f85a35fcce8e7a1b', source_code=['GFW-REVIEW', 'ICCAT', 'IMO', 'ISSF', 'NEAFC', 'RUS', 'TMT_OTHER'], ssvid='210631000', flag='CYP', ship_name='FRIO FORWIN', n_ship_name='FRIOFORWIN', call_sign='5BWC3', imo='9076260', latest_vessel_info=None, transmission_date_from=dat

In [19]:
vessels.data[-1].combined_sources_info

[CombinedSourceInfo(vessel_id='0edad163f-f53d-9ddb-5bef-f4a46be18559', gear_types=[GearType(name='CARRIER', source='GFW_VESSEL_LIST', year_from=2024, year_to=2025)], ship_types=[ShipType(name='CARRIER', source='GFW_VESSEL_LIST', year_from=2024, year_to=2025)]),
 CombinedSourceInfo(vessel_id='8c7304226-6c71-edbe-0b63-c246734b3c01', gear_types=[GearType(name='CARRIER', source='GFW_VESSEL_LIST', year_from=2013, year_to=2025)], ship_types=[ShipType(name='CARRIER', source='GFW_VESSEL_LIST', year_from=2013, year_to=2025)]),
 CombinedSourceInfo(vessel_id='0cb77880e-ee49-2ce4-a849-c0b413b6ec89', gear_types=[GearType(name='CARRIER', source='GFW_VESSEL_LIST', year_from=2012, year_to=2013)], ship_types=[ShipType(name='CARRIER', source='GFW_VESSEL_LIST', year_from=2012, year_to=2013)]),
 CombinedSourceInfo(vessel_id='da1cd7e1b-b8d0-539c-6581-2b3df8d0a6af', gear_types=[GearType(name='CARRIER', source='GFW_VESSEL_LIST', year_from=2015, year_to=2025)], ship_types=[ShipType(name='CARRIER', source='GFW

In [20]:
vessels.data[-1].model_dump()

{'dataset': 'public-global-vessel-identity:v3.0',
 'registry_info_total_records': 2,
 'registry_info': [{'id': 'a8d00ce54b37add7f85a35fcce8e7a1b',
   'source_code': ['GFW-REVIEW', 'IMO', 'NEAFC', 'TMT_OTHER'],
   'ssvid': '273379740',
   'flag': 'RUS',
   'ship_name': 'FRIO FORWIN',
   'n_ship_name': 'FRIOFORWIN',
   'call_sign': 'UCRZ',
   'imo': '9076260',
   'latest_vessel_info': True,
   'transmission_date_from': datetime.datetime(2015, 2, 27, 10, 59, 43, tzinfo=TzInfo(UTC)),
   'transmission_date_to': datetime.datetime(2025, 2, 27, 11, 22, 25, tzinfo=TzInfo(UTC)),
   'gear_types': ['SPECIALIZED_REEFER'],
   'length_m': 126.8,
   'tonnage_gt': 6963.6,
   'vessel_info_reference': '186aeab79b33d7f5560894c4ad53a0b1',
   'extra_fields': [{'registry_source': 'TMT',
     'iuu_status': None,
     'has_compliance_info': None,
     'images': [],
     'operator': None,
     'builtYear': None,
     'depthM': None}]},
  {'id': 'a8d00ce54b37add7f85a35fcce8e7a1b',
   'source_code': ['GFW-REVIEW'

### Get vessel by id

#### Define request parameters

In [21]:
dataset = VesselDataset.VESSEL_IDENTITY_LATEST
id = "8c7304226-6c71-edbe-0b63-c246734b3c01"

#### Get a vessel by its id

In [22]:
vessel = await client.vessels.aget_vessel(id=id, dataset=dataset)

In [23]:
vessel

In [24]:
vessel.data.registry_info

[RegistryInfo(id='a8d00ce54b37add7f85a35fcce8e7a1b', source_code=['GFW-REVIEW', 'IMO', 'NEAFC', 'TMT_OTHER'], ssvid='273379740', flag='RUS', ship_name='FRIO FORWIN', n_ship_name='FRIOFORWIN', call_sign='UCRZ', imo='9076260', latest_vessel_info=True, transmission_date_from=datetime.datetime(2015, 2, 27, 10, 59, 43, tzinfo=TzInfo(UTC)), transmission_date_to=datetime.datetime(2025, 2, 27, 11, 22, 25, tzinfo=TzInfo(UTC)), gear_types=['SPECIALIZED_REEFER'], length_m=126.8, tonnage_gt=6963.6, vessel_info_reference='186aeab79b33d7f5560894c4ad53a0b1', extra_fields=[ExtraField(registry_source='TMT', iuu_status=None, has_compliance_info=None, images=[], operator=None, builtYear=None, depthM=None)]),
 RegistryInfo(id='a8d00ce54b37add7f85a35fcce8e7a1b', source_code=['GFW-REVIEW', 'ICCAT', 'IMO', 'ISSF', 'NEAFC', 'RUS', 'TMT_OTHER'], ssvid='210631000', flag='CYP', ship_name='FRIO FORWIN', n_ship_name='FRIOFORWIN', call_sign='5BWC3', imo='9076260', latest_vessel_info=None, transmission_date_from=dat

In [25]:
vessel.data.combined_sources_info

[CombinedSourceInfo(vessel_id='0edad163f-f53d-9ddb-5bef-f4a46be18559', gear_types=[GearType(name='CARRIER', source='GFW_VESSEL_LIST', year_from=2024, year_to=2025)], ship_types=[ShipType(name='CARRIER', source='GFW_VESSEL_LIST', year_from=2024, year_to=2025)]),
 CombinedSourceInfo(vessel_id='8c7304226-6c71-edbe-0b63-c246734b3c01', gear_types=[GearType(name='CARRIER', source='GFW_VESSEL_LIST', year_from=2013, year_to=2025)], ship_types=[ShipType(name='CARRIER', source='GFW_VESSEL_LIST', year_from=2013, year_to=2025)]),
 CombinedSourceInfo(vessel_id='0cb77880e-ee49-2ce4-a849-c0b413b6ec89', gear_types=[GearType(name='CARRIER', source='GFW_VESSEL_LIST', year_from=2012, year_to=2013)], ship_types=[ShipType(name='CARRIER', source='GFW_VESSEL_LIST', year_from=2012, year_to=2013)]),
 CombinedSourceInfo(vessel_id='da1cd7e1b-b8d0-539c-6581-2b3df8d0a6af', gear_types=[GearType(name='CARRIER', source='GFW_VESSEL_LIST', year_from=2015, year_to=2025)], ship_types=[ShipType(name='CARRIER', source='GFW

In [26]:
vessel.data.model_dump()

{'dataset': 'public-global-vessel-identity:v3.0',
 'registry_info_total_records': 5,
 'registry_info': [{'id': 'a8d00ce54b37add7f85a35fcce8e7a1b',
   'source_code': ['GFW-REVIEW', 'IMO', 'NEAFC', 'TMT_OTHER'],
   'ssvid': '273379740',
   'flag': 'RUS',
   'ship_name': 'FRIO FORWIN',
   'n_ship_name': 'FRIOFORWIN',
   'call_sign': 'UCRZ',
   'imo': '9076260',
   'latest_vessel_info': True,
   'transmission_date_from': datetime.datetime(2015, 2, 27, 10, 59, 43, tzinfo=TzInfo(UTC)),
   'transmission_date_to': datetime.datetime(2025, 2, 27, 11, 22, 25, tzinfo=TzInfo(UTC)),
   'gear_types': ['SPECIALIZED_REEFER'],
   'length_m': 126.8,
   'tonnage_gt': 6963.6,
   'vessel_info_reference': '186aeab79b33d7f5560894c4ad53a0b1',
   'extra_fields': [{'registry_source': 'TMT',
     'iuu_status': None,
     'has_compliance_info': None,
     'images': [],
     'operator': None,
     'builtYear': None,
     'depthM': None}]},
  {'id': 'a8d00ce54b37add7f85a35fcce8e7a1b',
   'source_code': ['GFW-REVIEW'

## Events API

[Events API](https://globalfishingwatch.org/our-apis/documentation#events-api) allows you to explore different activities of a vessel. You can analyze:

- apparent fishing events
- encounters between these vessel types: fishing-carrier, fishing-support, fishing-bunker, fishing-fishing, tanker-fishing, carrier-bunker and support-banker.
- loitering for all vessel types
- port visits for all vessel types
- AIS off (aka GAPs) for all vessel types

Check more detail about data caveats [here](https://globalfishingwatch.org/our-apis/documentation#data-caveat).


### Import necessary data types

In [27]:
from gfwapiclient.resources.events.base.enums import EventDataset

### Get All Events

#### Define request parameters

In [28]:
datasets = [EventDataset.FISHING_EVENTS_LATEST]
start_date = datetime.date.fromisoformat("2017-01-01")
end_date = datetime.date.fromisoformat("2017-01-31")
limit = 1
offset = 0

#### Get all events

In [29]:
events = await client.events.aget_events(
    datasets=datasets,
    start_date=start_date,
    end_date=end_date,
    limit=limit,
    offset=offset,
)

In [30]:
events

In [31]:
events.data[-1]

EventListItem(start=datetime.datetime(2016, 12, 17, 20, 57, 49, tzinfo=TzInfo(UTC)), end=datetime.datetime(2017, 1, 4, 17, 27, 1, tzinfo=TzInfo(UTC)), id='3ca9b73aee21fbf278a636709e0f8f03', type='fishing', position=EventPosition(lat=54.9204, lon=-5.5465), regions=EventRegions(mpa=['555624876', '555786438', '555786428', '555557202'], eez=['5696'], rfmo=['ICCAT', 'NASCO', 'ICES', 'NEAFC', 'ACAP', 'IWC', 'NAMMCO'], fao=['27.6', '27', '27.6.a', '27.7.a', '27.7'], major_fao=['27'], eez_12_nm=['5696'], high_seas=[], mpa_no_take_partial=[], mpa_no_take=[]), bounding_box=[-5.3292866667, 54.775575, -5.943021666699991, 55.0624116667], distances=EventDistances(start_distance_from_shore_km=14.0, end_distance_from_shore_km=2.0, start_distance_from_port_km=15.019969999999999, end_distance_from_port_km=7.348033999999999), vessel=EventVessel(id='126570026-6217-2a10-f383-a42f66f78ea7', name='NORTHERN VIKING', ssvid='235063795', flag='GBR', type='fishing', public_authorizations=[EventVesselPublicAuthori

In [32]:
events.data[-1].model_dump()

{'start': datetime.datetime(2016, 12, 17, 20, 57, 49, tzinfo=TzInfo(UTC)),
 'end': datetime.datetime(2017, 1, 4, 17, 27, 1, tzinfo=TzInfo(UTC)),
 'id': '3ca9b73aee21fbf278a636709e0f8f03',
 'type': 'fishing',
 'position': {'lat': 54.9204, 'lon': -5.5465},
 'regions': {'mpa': ['555624876', '555786438', '555786428', '555557202'],
  'eez': ['5696'],
  'rfmo': ['ICCAT', 'NASCO', 'ICES', 'NEAFC', 'ACAP', 'IWC', 'NAMMCO'],
  'fao': ['27.6', '27', '27.6.a', '27.7.a', '27.7'],
  'major_fao': ['27'],
  'eez_12_nm': ['5696'],
  'high_seas': [],
  'mpa_no_take_partial': [],
  'mpa_no_take': []},
 'bounding_box': [-5.3292866667, 54.775575, -5.943021666699991, 55.0624116667],
 'distances': {'start_distance_from_shore_km': 14.0,
  'end_distance_from_shore_km': 2.0,
  'start_distance_from_port_km': 15.019969999999999,
  'end_distance_from_port_km': 7.348033999999999},
 'vessel': {'id': '126570026-6217-2a10-f383-a42f66f78ea7',
  'name': 'NORTHERN VIKING',
  'ssvid': '235063795',
  'flag': 'GBR',
  'typ

### Get one by Event ID

#### Define request parameters

In [33]:
id = "3ca9b73aee21fbf278a636709e0f8f03"
dataset = EventDataset.FISHING_EVENTS_LATEST

#### Get event by ID

In [34]:
event = await client.events.aget_event(id=id, dataset=dataset)

In [35]:
event

In [36]:
event.data.id

'3ca9b73aee21fbf278a636709e0f8f03'

In [37]:
event.data.distances

EventDistances(start_distance_from_shore_km=14.0, end_distance_from_shore_km=2.0, start_distance_from_port_km=15.019969999999999, end_distance_from_port_km=7.348033999999999)

In [38]:
event.data.model_dump()

{'start': datetime.datetime(2016, 12, 17, 20, 57, 49, tzinfo=TzInfo(UTC)),
 'end': datetime.datetime(2017, 1, 4, 17, 27, 1, tzinfo=TzInfo(UTC)),
 'id': '3ca9b73aee21fbf278a636709e0f8f03',
 'type': 'fishing',
 'position': {'lat': 54.9204, 'lon': -5.5465},
 'regions': {'mpa': ['555624876', '555786438', '555786428', '555557202'],
  'eez': ['5696'],
  'rfmo': ['ICCAT', 'NASCO', 'ICES', 'NEAFC', 'ACAP', 'IWC', 'NAMMCO'],
  'fao': ['27.6', '27', '27.6.a', '27.7.a', '27.7'],
  'major_fao': ['27'],
  'eez_12_nm': ['5696'],
  'high_seas': [],
  'mpa_no_take_partial': [],
  'mpa_no_take': []},
 'bounding_box': [-5.3292866667, 54.775575, -5.943021666699991, 55.0624116667],
 'distances': {'start_distance_from_shore_km': 14.0,
  'end_distance_from_shore_km': 2.0,
  'start_distance_from_port_km': 15.019969999999999,
  'end_distance_from_port_km': 7.348033999999999},
 'vessel': {'id': '126570026-6217-2a10-f383-a42f66f78ea7',
  'name': 'NORTHERN VIKING',
  'ssvid': '235063795',
  'flag': 'GBR',
  'typ

### Statistics on events

Get events statistics worldwide or for a specific region.

#### Define request parameters

In [39]:
datasets = [EventDataset.FISHING_EVENTS_LATEST]
start_date = datetime.date.fromisoformat("2018-01-01")
end_date = datetime.date.fromisoformat("2023-01-31")

#### Get event statistics

In [40]:
event_stats = await client.events.aget_event_stats(
    datasets=datasets,
    start_date=start_date,
    end_date=end_date,
)

In [41]:
event_stats

In [42]:
event_stats.data.num_events, event_stats.data.num_flags, event_stats.data.num_vessels

(27628441, 228, 290857)

In [43]:
event_stats.data.timeseries

[EventStatsTimeSeries(date=datetime.datetime(2018, 1, 1, 0, 0, tzinfo=TzInfo(UTC)), value=4462237),
 EventStatsTimeSeries(date=datetime.datetime(2019, 1, 1, 0, 0, tzinfo=TzInfo(UTC)), value=4791179),
 EventStatsTimeSeries(date=datetime.datetime(2020, 1, 1, 0, 0, tzinfo=TzInfo(UTC)), value=4883901),
 EventStatsTimeSeries(date=datetime.datetime(2021, 1, 1, 0, 0, tzinfo=TzInfo(UTC)), value=5267339),
 EventStatsTimeSeries(date=datetime.datetime(2022, 1, 1, 0, 0, tzinfo=TzInfo(UTC)), value=7615881),
 EventStatsTimeSeries(date=datetime.datetime(2023, 1, 1, 0, 0, tzinfo=TzInfo(UTC)), value=605760)]

In [44]:
event_stats.data.model_dump()

{'num_events': 27628441,
 'num_flags': 228,
 'num_vessels': 290857,
 'flags': ['',
  'ARM',
  'VIR',
  'TCA',
  'SAU',
  'URY',
  'SOM',
  'JOR',
  'KAZ',
  'BGD',
  'TGO',
  'DMA',
  'IDN',
  'KHM',
  'SYC',
  'VEN',
  'AIA',
  'ESP',
  'BTN',
  'MLT',
  'TWN',
  'CYP',
  'FRO',
  'ROU',
  'HTI',
  'CUB',
  'BLR',
  'MYS',
  'GRD',
  'BDI',
  'TUV',
  'ABW',
  'MNP',
  'MWI',
  'BRA',
  'HUN',
  'GRL',
  'KOR',
  'ZAF',
  'AFG',
  'PNG',
  'BEN',
  'LBN',
  'PHL',
  'RUS',
  'BFA',
  'NIU',
  'SSD',
  'TZA',
  'DJI',
  'MDG',
  'TJK',
  'THA',
  'GBR',
  'AUT',
  'DNK',
  'USA',
  'NOR',
  'VCT',
  'CAN',
  'SWZ',
  'AZE',
  'PYF',
  'KGZ',
  'SRB',
  'QAT',
  'FJI',
  'CCK',
  'IRN',
  'NIC',
  'KWT',
  'IRQ',
  'CIV',
  'ISL',
  'MTQ',
  'PRK',
  'TUN',
  'PRI',
  'BGR',
  'MNE',
  'SDN',
  'LSO',
  'SMR',
  'TUR',
  'TKM',
  'PRT',
  'NLD',
  'NPL',
  'GHA',
  'GTM',
  'DZA',
  'MUS',
  'BHR',
  'CAF',
  'EST',
  'BHS',
  'BOL',
  'YEM',
  'KIR',
  'COL',
  'GEO',
  'CPV',
  'DOM',

## Insights API

The [Insights API](https://globalfishingwatch.org/our-apis/documentation#insights-api) is a set of indicators or `vessel insights` that bring together important information on a vessel's known activity (based on AIS), vessel identity information and public authorizations. The objective of the insights is to support risk-based decision-making, operational planning, and other due diligence activities by making it easier for a user to identify vessel characteristics that can indicate an increased potential or opportunity for a vessel to engage in IUU (Illegal, Unreported, or Unregulated) fishing.


### Insights by vessels

This endpoint allow to get vessel insights for one or several vessels

#### Import necessary data types

In [45]:
from gfwapiclient.resources.insights.models.request import (
    VesselInsightIdBody,
    VesselInsightInclude,
)

#### Define request parameters

In [46]:
start_date = datetime.date.fromisoformat("2020-01-01")
end_date = datetime.date.fromisoformat("2025-03-03")

vessels = [
    VesselInsightIdBody(
        dataset_id="public-global-vessel-identity:latest",
        vessel_id="2d26aa452-2d4f-4cae-2ec4-377f85e88dcb",
    ),
]

#### Get any apparent fishing insights of vessels

In [47]:
includes = [
    VesselInsightInclude.FISHING,
]

In [48]:
fishing_insights = await client.insights.aget_vessel_insights(
    includes=includes,
    start_date=start_date,
    end_date=end_date,
    vessels=vessels,
)

In [49]:
fishing_insights

In [50]:
fishing_insights.data.model_dump()

{'period': {'start_date': datetime.date(2020, 1, 1),
  'end_date': datetime.date(2025, 3, 3)},
 'vessel_ids_without_identity': None,
 'gap': None,
 'coverage': None,
 'apparent_fishing': {'datasets': ['public-global-fishing-events:v3.0'],
  'historical_counters': None,
  'period_selected_counters': {'events': 0,
   'events_gap_off': None,
   'events_in_rfmo_without_known_authorization': 0,
   'events_in_no_take_mpas': 0},
  'events_in_rfmo_without_known_authorization': [],
  'events_in_no_take_mpas': []},
 'vessel_identity': None}

#### Get being present on an RFMO IUU vessel list insights of vessels

In [51]:
includes = [
    VesselInsightInclude.VESSEL_IDENTITY_IUU_VESSEL_LIST,
]

In [52]:
iuu_insights = await client.insights.aget_vessel_insights(
    includes=includes,
    start_date=start_date,
    end_date=end_date,
    vessels=vessels,
)

In [53]:
iuu_insights

In [54]:
iuu_insights.data.model_dump()

{'period': {'start_date': datetime.date(2020, 1, 1),
  'end_date': datetime.date(2025, 3, 3)},
 'vessel_ids_without_identity': None,
 'gap': None,
 'coverage': None,
 'apparent_fishing': None,
 'vessel_identity': {'datasets': ['public-global-vessel-identity:v3.0'],
  'iuu_vessel_list': {'values_in_the_period': [],
   'total_times_listed': 0,
   'total_times_listed_in_the_period': 0}}}

#### Get AIS off insights of vessels

In [55]:
includes = [
    VesselInsightInclude.GAP,
]

In [56]:
gap_insights = await client.insights.aget_vessel_insights(
    includes=includes,
    start_date=start_date,
    end_date=end_date,
    vessels=vessels,
)

In [57]:
gap_insights

In [58]:
gap_insights.data.model_dump()

{'period': {'start_date': datetime.date(2020, 1, 1),
  'end_date': datetime.date(2025, 3, 3)},
 'vessel_ids_without_identity': None,
 'gap': {'datasets': ['public-global-gaps-events:v3.0'],
  'historical_counters': None,
  'period_selected_counters': {'events': 0,
   'events_gap_off': 0,
   'events_in_rfmo_without_known_authorization': None,
   'events_in_no_take_mpas': None},
  'ais_off': []},
 'coverage': None,
 'apparent_fishing': None,
 'vessel_identity': None}

#### Get AIS coverage metric insights of vessels

In [59]:
includes = [
    VesselInsightInclude.COVERAGE,
]

In [60]:
coverage_insights = await client.insights.aget_vessel_insights(
    includes=includes,
    start_date=start_date,
    end_date=end_date,
    vessels=vessels,
)

In [61]:
coverage_insights

In [62]:
coverage_insights.data.model_dump()

{'period': {'start_date': datetime.date(2020, 1, 1),
  'end_date': datetime.date(2025, 3, 3)},
 'vessel_ids_without_identity': None,
 'gap': None,
 'coverage': {'blocks': 4599,
  'blocks_with_positions': 4044,
  'percentage': 87.93215916503587},
 'apparent_fishing': None,
 'vessel_identity': None}

#### Get all insights of vessels

In [63]:
includes = [
    VesselInsightInclude.GAP,
    VesselInsightInclude.COVERAGE,
    VesselInsightInclude.FISHING,
    VesselInsightInclude.VESSEL_IDENTITY_IUU_VESSEL_LIST,
]

In [64]:
all_insights = await client.insights.aget_vessel_insights(
    includes=includes,
    start_date=start_date,
    end_date=end_date,
    vessels=vessels,
)

In [65]:
all_insights

In [66]:
all_insights.data.model_dump()

{'period': {'start_date': datetime.date(2020, 1, 1),
  'end_date': datetime.date(2025, 3, 3)},
 'vessel_ids_without_identity': None,
 'gap': {'datasets': ['public-global-gaps-events:v3.0'],
  'historical_counters': None,
  'period_selected_counters': {'events': 0,
   'events_gap_off': 0,
   'events_in_rfmo_without_known_authorization': None,
   'events_in_no_take_mpas': None},
  'ais_off': []},
 'coverage': {'blocks': 4599,
  'blocks_with_positions': 4044,
  'percentage': 87.93215916503587},
 'apparent_fishing': {'datasets': ['public-global-fishing-events:v3.0'],
  'historical_counters': None,
  'period_selected_counters': {'events': 0,
   'events_gap_off': None,
   'events_in_rfmo_without_known_authorization': 0,
   'events_in_no_take_mpas': 0},
  'events_in_rfmo_without_known_authorization': [],
  'events_in_no_take_mpas': []},
 'vessel_identity': {'datasets': ['public-global-vessel-identity:v3.0'],
  'iuu_vessel_list': {'values_in_the_period': [],
   'total_times_listed': 0,
   'tot

## Reference Data API

### Regions

The `Events API` and `4Wings API - Report` uses regions ids. In order to get the details of those regions, call the endpoints described below.

#### Get list of Exclusive economic zones (EEZs)

In [67]:
eez_regions = await client.references.aget_eez_regions()

In [68]:
eez_regions

In [69]:
eez_regions.data[-1]

EEZRegionItem(id=26523, label='Turkmenistan', iso3='TKM', dataset='public-eez-areas')

#### Get list of Marine protected areas (MPAs)

In [70]:
mpa_regions = await client.references.aget_mpa_regions()

In [71]:
mpa_regions

In [72]:
mpa_regions.data[-1]

MPARegionItem(id='555790281', label='Iles Barren - Aire protegee', name='Iles Barren', dataset='public-mpa-all')

#### Get list of Regional fisheries management organization (RFMO)

In [73]:
rfmo_regions = await client.references.aget_rfmo_regions()

In [74]:
rfmo_regions

In [75]:
rfmo_regions.data[-1]

RFMORegionItem(id='WCPFC', label='WCPFC', rfb=None, dataset='public-rfmo')